Current questions on development: 

- Should we include the first (embedding) layer in energy computations? I don't want to leave out model information but it leads to NaNs in the padding tokens since they generate the same activations in this layer -> null vectors -> 0 dot product
- Should we do an if case for when the norm of the vector is 0 in the angle computation? Or is NaN more approriate (right now I think the second, because having an angle of 0 has a meaning that in this case wouldn't be the same) 

In [ ]:
import transformers
import torch
import json
from math import acos

In [ ]:
from LLMfunctions import inference_activations

## Recreate GPT-2XL set-up for Llama

In [3]:
prompt_topic = 'viktor'
prompt_sufix = '_' + prompt_topic
with open('prompts-gen/'+prompt_topic+'.txt') as file:
    prompt = file.read()
prompt = json.loads(prompt, strict=False) #transform string to dict ready for model; strict ignores space characters

In [4]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
    
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id, padding_side = "left") #choose where padding will be applioed
tokenizer.pad_token_id = tokenizer.eos_token_id #required in llama because no padding token is defined
model = transformers.AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
text = tokenizer.apply_chat_template(prompt, add_generation_prompt=True, tokenize=False) #prompt-adds token when the model should generate; tokenize- if we should tokenize the output, rn will be a string
inputs = tokenizer(text, padding="longest", return_tensors="pt") #transform into pt (pytorch) tensors; pad to the longest sequence in the batch
inputs = {key: val.cuda() for key, val in inputs.items()} #move inputs into cuda
temp_texts=tokenizer.batch_decode(inputs["input_ids"], skip_special_tokens=True) #way to debug inputs

In [6]:
num_generations = 5  

generations = model.generate(
    **inputs,
    max_new_tokens=400,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=terminators,
    num_return_sequences=num_generations  
)

In [7]:
prompt_text = tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True)
decoded_gens = tokenizer.batch_decode(generations, skip_special_tokens=True)
decoded_stories = [tokens[len(prompt_text):] for tokens in decoded_gens]

Bugs I found:
- We are passing through the pipeline the generation including the prompt. We need to find the number of tokens in the tensor corresponding to the prompt - this has to be done in the forward pass. - done 

## Fixing angle computation 

In [113]:
#legacy pipepline

def compute_layer_vectors(layer_activation):
    return layer_activation[1:]-layer_activation[:-1] #matrix except first_row - matrix except last_row

def compute_vectors(hidden_states):
    return [compute_layer_vectors(layer) for layer in hidden_states]

def compute_dot_product_layer(layer_vectors):
    return torch.stack([torch.dot(layer_vectors[i,:],layer_vectors[i+1,:]) for i in range(layer_vectors.shape[0]-1)]) #these are 1D vectors so keep torch.dot

def compute_dot_product(vector_transitions_trajectory):
    return [compute_dot_product_layer(layer_vectors) for layer_vectors in vector_transitions_trajectory]  

def average_layer_dot_product(layer_dot_product):
    # return layer_dot_product.nanmean()
    return layer_dot_product.mean()

def average_dot_product(dot_product_list):
    return torch.stack([average_layer_dot_product(layer_dot_product) for layer_dot_product in dot_product_list])

def sum_layer_energy(average_layer_dot_product):
    return average_layer_dot_product.sum()

def energy_pipeline(layer_hidden_states):
    if not isinstance(layer_hidden_states, list):
        raise TypeError("Expected a list of tensors (one per layer + embedding layer).")
    return sum_layer_energy(average_dot_product(compute_dot_product(compute_vectors(layer_hidden_states)))).item()

In [114]:
#new pipeline

def compute_layer_vectors(layer_activation):
    return layer_activation[1:]-layer_activation[:-1] #matrix except first_row - matrix except last_row

def compute_vectors(hidden_states):
    return [compute_layer_vectors(layer) for layer in hidden_states]

def compute_angle_layer(layer_vectors):
    angles=[]

    for i in range(layer_vectors.shape[0]-1):
        a = layer_vectors[i,:]
        b = layer_vectors[i+1,:]

        # if torch.norm(a) == 0 or torch.norm(b) == 0. #doesnt really make sense to do this
        #     angles.append(0)

        angles.append(acos(torch.dot(a, b) / (torch.norm(a) * torch.norm(b))))

    return torch.tensor(angles, dtype=torch.bfloat16)

def compute_angle(vector_transitions_trajectory):
    return [compute_angle_layer(layer_vectors) for layer_vectors in vector_transitions_trajectory]

def average_layer_angle(layer_dot_product):
    # return layer_dot_product.nanmean()
    return layer_dot_product.nanmean()

def average_angle(dot_product_list):
    return torch.stack([average_layer_angle(layer_dot_product) for layer_dot_product in dot_product_list])

def sum_layer_energy(average_layer_dot_product):
    return average_layer_dot_product.sum()

def energy_pipeline_angles(layer_hidden_states):
    if not isinstance(layer_hidden_states, list):
        raise TypeError("Expected a list of tensors (one per layer + embedding layer).")
    return sum_layer_energy(average_angle(compute_angle(compute_vectors(layer_hidden_states)))).item()

In [110]:
tensor_size_prompt = inputs['input_ids'].shape[1]
first_gen_withtout_prompt = generations[0,tensor_size_prompt:].unsqueeze(0)

In [111]:
energy_values = []
for i in range(num_generations):
    tensor = generations[i,tensor_size_prompt:].unsqueeze(0) #shape 1xseq_length; remove prompt tokens
    activations = inference_activations(model,tensor)
    energy_values.append(energy_pipeline_angles(activations))

In [112]:
energy_values #nan, nan, 65, nan, nan

[60.5, 64.5, 65.0, 60.0, 59.5]

#### Debugging pipeline - NaN energy values

**Bug**: N-1 stories in batch return NaN values.

**Reason**: The last tokens in most of the stories correspond to the padding token, which leads to the same activations in the first layer (which job is to embed). In return, the vectors of consecutive positions are null which lead to NaNs when computing the angle between them.

**Debugging task**: Make sure that the only story which doesn't return NaN is the largest (no padding) in the number of tokens. 

- **Hypothesis**: The largest story  is the only story that doesn't need padding at the end of the sentence so it doesn't repeat tokens. Therefore there's no vectors equal to zero which are the cause of NaNs.

In [107]:
for i in range(len(generations)):
    if generations[i,-2] != 128009: #padding token
        print(f"The story at index {i} is the longest therefore it shouldn't include NaNs.")

The story at index 2 is the longest therefore it shouldn't include NaNs.


In [108]:
for i in range(len(generations)):
    print(f'Story with index {i} has an energy value of {energy_values[i]}')

Story with index 0 has an energy value of nan
Story with index 1 has an energy value of nan
Story with index 2 has an energy value of 65.0
Story with index 3 has an energy value of nan
Story with index 4 has an energy value of nan
